In [52]:
using DataFrames
ped  = readtable("LICPedigree.seq",separator=' ')
n   = size(ped,1)

1148677

In [56]:
aij = Dict{Int64, Float64}()
fArray = fill(-1.0,n)
head(ped)

,ind,sire,dam
1,1,0,0
2,2,0,0
3,3,0,0
4,4,2,3
5,5,0,0
6,6,4,5


In [45]:
function calcAddRel!(aij::Dict{Int64, Float64},ped::DataFrame,id1::Int64,id2::Int64)
    if id1==0 || id2==0           
        return 0.0
    end
    old,yng = id1<id2 ? (id1,id2):(id2,id1)
    n = yng - 1
    
    aijKey = n*(n+1)/2 + old 
    if haskey(aij,aijKey)     
        return aij[aijKey]
    end
    
    sireOfYng = ped[yng,:sire]
    damOfYng  = ped[yng,:dam] 
    if old==yng                       
        aii = 1 + 0.5*calcAddRel!(aij,ped,damOfYng,sireOfYng) 
        aij[aijKey] = aii
        return (aii)   
    end
    aOldDamYoung  = (old==0 || damOfYng ==0)? 0.0:calcAddRel!(aij,ped,old,damOfYng)
    aOldSireYoung = (old==0 || sireOfYng==0)? 0.0:calcAddRel!(aij,ped,old,sireOfYng)
    aijVal = 0.5*(aOldSireYoung + aOldDamYoung)
    aij[aijKey] = aijVal
    return(aijVal)
end 

calcAddRel! (generic function with 2 methods)

In [46]:
function calcInbreeding!(fArray::Array{Float64,1},aij::Dict{Int64, Float64},ped::DataFrame,
    id::Int64)
    if fArray[id] > -1.0
        return fArray[id]
    end
    sireID = ped[id,:sire]
    damID  = ped[id,:dam ]
    if (sireID==0 || damID==0 )
        fArray[id] = 0.0
    else
        fArray[id] = 0.5*calcAddRel!(aij,ped,sireID,damID)
    end
end

calcInbreeding! (generic function with 2 methods)

In [47]:
function testSpeed(n,fArray,aij,ped)
    for ind in 1:n
        calcInbreeding!(fArray,aij,ped,ind)
    end
end

testSpeed (generic function with 1 method)

In [57]:
@time testSpeed(n,fArray,aij,ped)

  1.239041 seconds (11.26 M allocations: 304.989 MB, 8.45% gc time)


In [58]:
sortrows([fArray ped[:ind]])

1148677x2 Array{Float64,2}:
 0.0            1.0      
 0.0            2.0      
 0.0            3.0      
 0.0            4.0      
 0.0            5.0      
 0.0            6.0      
 0.0            7.0      
 0.0            8.0      
 0.0            9.0      
 0.0           10.0      
 0.0           11.0      
 0.0           12.0      
 0.0           13.0      
 ⋮                       
 0.265625       1.08718e6
 0.265625       1.14423e6
 0.28125   835126.0      
 0.28125        1.09405e6
 0.28125        1.13339e6
 0.3125    250961.0      
 0.3125    281308.0      
 0.3125    319881.0      
 0.3125    790440.0      
 0.3125    810528.0      
 0.3125    921567.0      
 0.3125    950010.0      